# Chapter 8. Communication Between Django and Tornado
we'll expand on that server to allow the Django app to push updates, improve the security of websocket connection.
Redis as a message broker to improve the future scalability of the server.

* [Websocket 개념 참조](http://d2.naver.com/helloworld/1336)
* [Publish/Subsrcibe 모델](http://arsviator.blogspot.kr/2015/03/mqttmessage-queue-telemetry-transport.html)
* [Redis 개념](http://bcho.tistory.com/654)

## Receiving Updates in Tornado
Tornado server manages all of its client subscriptions & broadcasts internally. No way broadcast a message from outside Tornado.

#### Redis
 * as a message broker for this broadcast.
 * app to know the format of messages & how th messages are broadcast by Tornado.
 
**[Code Changed] :**
     watercooler.py
 
     - create UpdateHandler
     - add routes related UpdateHandler in ScrumApplication class.
 

###  Sending Updates from Django
**post_save, pre_delete** hooks priovided by **django-rest-framework**, Since this update will be added to all of the API views, it will be implemented as a mixin class in board/views.py

**[Code Changed] :**
     board/views.py
 
     create UpdateHookMixin class


It needs the pre_delete rather than post_delete so that the object pk will still be available.
There is a short timeout to prevent the hook from blocking the API response for too long.

**[Code Changed] :**
     board/views.py
 
     add UpdateHookMixin class definition

Adavantage|Disadvantage
----------|------------
clients can receive updatess created outside the API, such as background jobs.|add overhead to each model save by potentially broadcasting changes when no client is listening.

[참조] [Mixin class 개념](https://en.wikipedia.org/wiki/Mixin)

### Handling Updates on the Client
Django app will notify the Tornado server when there are model changes, and **Tornado will broadcast those changes to all connected clients.**

**[Code Changed] :**
     board/static/board/js/views.js
 
     add 'task:add','task:update','task:remove' events

This is inefficient for the client that made the original addition or edit, since it will refetch the task information it already has.

**It also means that all clients will make the same API call around the same time, creating a [mini-denial-of-service](https://ko.wikipedia.org/wiki/%EC%84%9C%EB%B9%84%EC%8A%A4_%EA%B1%B0%EB%B6%80_%EA%B3%B5%EA%B2%A9) attack on our own server.**

## Server Improvements

###1. Robust Subscriptions
To resolve this issue, **the server will use Redis as a message broker. Redis is a popular key-value store. that also has support for pub-sub channels.** (Redis isn't true message broker like RabbitMQ or ActiveMQ)

Interacting with Redis will require the tornado-redis libray from Pypi.

```
    # pip install tornado-redis redis
```
tornado-redis has built-in class for managing a pub-sub channel when you are using SockJS or Socket.IO.

**[Code Changed] :**
     wartercooler.py
 
     - create RedisSubscriber extension of BaseSubscriber
     - modify ScrumApplication 
       * async client for managing the subscription
       * sync one for broadcasting new message
       

ScumApplication no longer know names of all of the channels in the case of multiple websocket servers. So the use of all channel is required.
The JS client put an identifier in the message body so that it can choose to ignore messages that orginated from itself.

**[Code Changed] :**
     wartercooler.py
 
     - SprintHandler class open method
       * generate a unique ID when the connection is opened.
     - ScrumApplication class
       * message is wrapped to annotate original sender.
     - modify RedisHandler, on_message
       * If there was no sender, the message is sent to all subscribers.
       * If means sender == subscriber.uid then, message is sent to all subscribers
       

* **Tornado server can be scaled beyond a single process by making use of Redis as a simple message broker.**
* the server could be made more secure.

### 2. Websocket Authentication
서버에 아무 메세지를 보내거나, 잘못된 메세지를 보내더라도 message forwarding이 발생한다. Adding validation will server to authenticate the clients to prevent misuse.
이를 없애기 위해서, **두 서버사이에 정보를 전달할 수 있도록 secret key를 django와 websocket 서버에서 secret key를 공유한다.**

**[Code Changed] :**
     
     - scrum/settings.py
       * add "WATERCOOLER_SECRET" key
     - board/serializers.py
       * SprintSerializer class get-links method, add signer, channel
       
Sign을 하기 때문에 client에서 변조되는 것을 막을 수 있다.
django App에서 websocket 서버로 전송할때 Sprint의 obj.pk를 암호화 서명을 수행했기 때문에, Tornado Server에서 복호화를 하는 작업이 필요하다.

**[Code Changed] :**
     watercooler.py
     
     - open()
       * channel을 복호화하여 obj.pk를 얻어온다. 이 토큰은 30분이 지나면 expired 된다.
     - on_message(), on_close()
       * sprint가 존재여부 체크해서 있는 경우만 broadcast or remove
     - ScrumApplication
        * Tornado 서버에서 django app에서 사용하는 동일한 secret key를 암호화 한다.

현재 생성된 토큰이 소멸되는데 30분이 걸린다. Ideally this could be shorter, because the token is generated when the sprint is fetched on the home page, there might be a long delay between it is used on the sprint detail page.
**By making another API call to generate the channel URL right before it is needed. In that case, the timeout could be made shorter.**


[참조] [django cryptographic signing](https://docs.djangoproject.com/en/1.8/topics/signing/)

### 3. Better Updates
task가 드래그 앤 드랍되는 등의 변화가 API를 통해 전달되면, Django 서버는 Tornado 서버에게 update 되어야 하는 모든 client에게 전달하기 위해서 notify를 수행한다. Client 중에 브라우저가 있다면 브라우저는 API 콜을 수행한다. 만약 다른 client 들이 정보를 update 받기 위해서는 같은 API를 호출해야한다. 너무 비효율적이다.

이보다 효과적인 방법은 **send that information along to the clients through the websocket in the first place.**

#### Django  application needs to be updated to send current model state to the Tornado server.

**[Code Changed] :**
     
     board/view.py
     
     - _send_hook_request()
     
     watercooler.py
     - to be updated to send this data along to the client.
     - Parsed body data is sent along to the client through the websocket broadcase.
     
Critical issue with the Tornado webhook is that notihg secures this endpoint. Anywone with knowledge of the server location could push updates to all of the connected clients.  ** This won't directoly impact the database contents unless clients receive the bad model and then later save it.**



### 4. Secure Updates

Add a security layer outside the application
  * An HTTP proxy (ex. Nginx) could be placed in front of the server to enforce basic, diget, or client authentication
  * That authentication would need to be added to the request generated by the Django application.
  * 중앙 집중적인 처리가 가능, 개별 비지니스 컴포넌트에서 인증 처리를 할 때는 분산형 인증 처리가 되기 때문에 하나의 컴포넌트에서 인증한 정보를 다른 컴포넌트에서도 인증이 되어야 한다. ex) SSO
  
**It's also possible to include some verification of the messages between the two servers using the existing shared secret.
In the same way that channel is signed for verification, the webhook requests can be signed to ensure that they came from a trusted endpoint and were not modified in transit.**
  
[참조]
* [Reverse Proxy](http://bcho.tistory.com/661)
* [API Gateway](http://bcho.tistory.com/1005)